In [ ]:
# Import libraries

import sys
import logging
import pandas as pd
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.weather import load_weather
from library.network import build_network

logging.basicConfig(level=logging.INFO)

In [ ]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 5000
load_target = 15

In [ ]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [ ]:
network = build_network(index, resolution, geography, target_load, assumptions, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [ ]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

## Add battery charge/discharge ratio constraint
link_capacity = model.variables["Link-p_nom"]
lhs = link_capacity.loc["battery-charge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

In [ ]:
network.statistics().round(0)